# Impact of different preculturing methods on growth of E. coli#

### Notebook used to generate Supplementary Fig. S4 ###

In [1]:
#Import necessary modules

import impact as                             impt
import impact.plotting as                    implot
from impact.parsers import Parser as parser
import pandas as pd
import numpy as np
import sys
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"

if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML
    HTML("""
         <script>
          var waitForPlotly = setInterval( function() {
          if( typeof(window.Plotly) !== "undefined" ){
          MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
          MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
          clearInterval(waitForPlotly);}}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)

C:\Users\User\anaconda3\envs\impact\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Could not import pyexcel
No module named 'pyexcel_xlsx'


### Update impact Settings ###

In [2]:
impt.settings.perform_curve_fit = False
impt.settings.fit_type = 'gompertz'
impt.settings.outlier_cleaning_flag = False
impt.settings.max_fraction_replicates_to_remove = 0.5
impt.settings.std_deviation_cutoff=0.05
impt.settings.verbose = False
impt.settings.live_calculations = False
impt.settings.use_filtered_data = True

impt.settings.death_phase_hyperparameter = 4
impt.settings.savgolFilterWindowSize = 9

### Load data ###

In [3]:
data_format='spectromax_OD'
plate_type = '96 Wells'
file_name = 'PrecultureTest_01022021.xlsx'
expt = parser.parse_raw_data(data_format=data_format,file_name=file_name,plate_type=plate_type)
expt.calculate()



Importing data from PrecultureTest_01022021.xlsx...0.2s
Parsing time point list...Parsed 8160 time points in 1.1s
Parsing analyte list...Parsed 96 single trials in 521.9ms
Parsing single trial list...Parsed 15 replicates in 0.1s
Analyzing data...Ran analysis in 2.1s



### Plotting function ###

In [5]:
def growth_plot(replicate_trials=None, analyte='OD600', colors=None,
                        cl_scales=['8', 'qual', 'Set1'],
                        pts_per_hour=60,plot_start=0, plot_end=-1
                        ):
    traces = []
    colors = ['rgb(54, 146, 68)',
              'rgb(217, 72, 42)',
              'rgb(33, 100, 175)',
              'rgb(146, 54, 132)',
              'rgb(175, 127, 91)']
    fillcolors = ['rgba(54, 146, 68, 0.25)',
                  'rgba(217, 72, 42, 0.25)',
                  'rgba(33, 100, 175, 0.25)',
                  'rgba(146, 54, 132, 0.25)',
                 'rgba(175, 127, 91, 0.25)']
    replicate_trials = sorted(replicate_trials, key=lambda rep: str(rep.trial_identifier))
    
    for index, replicate in enumerate(replicate_trials):

        required_num_pts = replicate.t[plot_start:plot_end][-1] * pts_per_hour
        removePointFraction = int(len(replicate.t[plot_start:plot_end]) / required_num_pts)
        if removePointFraction < 1:  removePointFraction = 1
        time = replicate.t[plot_start:plot_end]

        avg_od = replicate.avg.analyte_dict[analyte].data_vector[plot_start:plot_end]
        std_od = replicate.std.analyte_dict[analyte].data_vector[plot_start:plot_end]
        shape='linear'

        feature = 'specific_productivity'
        avg_mu = getattr(replicate.avg.analyte_dict[analyte], feature)[plot_start:plot_end]
        std_mu = getattr(replicate.std.analyte_dict[analyte], feature)[plot_start:plot_end]
        shape='spline'
            
        y = avg_od[::removePointFraction]
        y_plus = avg_od[::removePointFraction] + std_od[::removePointFraction]
        y_minus = avg_od[::removePointFraction] - std_od[::removePointFraction]
        
        mu = avg_mu[::removePointFraction]
        mu_plus = avg_mu[::removePointFraction] + std_mu[::removePointFraction]
        mu_minus = avg_mu[::removePointFraction] - std_mu[::removePointFraction]

        traces.append(go.Scatter(x=time[::removePointFraction],
                                 y=y,
                                 mode='lines',
                                 line={'color':colors[index], 'shape':shape,'width':2.5},
                                 showlegend=True, yaxis='y2',
                                 legendgroup=str(replicate.trial_identifier),
                                 name=str(replicate.trial_identifier.media)+ ", DO: "+
                                      str(replicate.trial_identifier.environment.DO)))


        traces.append(go.Scatter(x=list(time[::removePointFraction])+list(time[::removePointFraction][::-1]),
                                 y=list(y_plus) + list(y_minus[::-1]),
                                 mode='lines',yaxis='y2',
                                 line={'color':'rgba(0,0,0,0)', 'shape':shape},
                                 fill='toself',
                                 fillcolor=fillcolors[index],
                                 showlegend=False,
                                 legendgroup=str(replicate.trial_identifier),
                                 name="error_"+str(replicate.trial_identifier.media)+ ", DO: "+
                                      str(replicate.trial_identifier.environment.DO)))
        
        
        traces.append(go.Scatter(x=time[::removePointFraction],
                         y=mu,
                         mode='lines',
                         marker={'color':colors[index]},
                         line={'color':colors[index], 'shape':shape,'width':2.5, 'dash':'dot'},
                         showlegend=False, yaxis='y1',
                         legendgroup=str(replicate.trial_identifier),
                         name=str(replicate.trial_identifier.media)+ ", DO: "+
                              str(replicate.trial_identifier.environment.DO)))


        traces.append(go.Scatter(x=list(time[::removePointFraction])+list(time[::removePointFraction][::-1]),
                                 y=list(mu_plus) + list(mu_minus[::-1]),
                                 mode='lines',
                                 line={'color':'rgba(0,0,0,0)', 'shape':shape},
                                 fill='toself',
                                 fillcolor=fillcolors[index],
                                 showlegend=False, yaxis='y1',
                                 legendgroup=str(replicate.trial_identifier),
                                 name="error_"+str(replicate.trial_identifier.media)+ ", DO: "+
                                      str(replicate.trial_identifier.environment.DO)))
        
        

    return traces


### Plot cell density and growth rates for different precultures ###

In [6]:
strain_list = ['WT', 'Δ(adh,pta)-D1', 'Δ(adh,pta)-D28', 'Δ(adh,pta)-D59']
pts_per_hour = 6
feature=None

layout = go.Layout(height=425, width=585, legend_x= 1.3,

                   xaxis=dict(title='Time (h)',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(0,8)),

                   yaxis2=dict(title='Cell Density (a.u. OD600)',
                              titlefont=dict(family='Myriad Pro', size=18, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, range=(0,1.1), tickvals=(0,0.25,0.5,0.75,1,1.25),
                              tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                              showgrid=True,zeroline=False,anchor='x',side='right'),
                  
                  yaxis=dict(title='Instantaneous Growth Rate (1/h)',
                            titlefont=dict(family='Myriad Pro', size=18, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False,
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(0, 2.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0, overlaying='y2'))


for strain in strain_list: 

    rep_list = [replicate for replicate in expt.replicate_trials if
                str(replicate.trial_identifier.strain.parent) == strain]

    rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier))

    tracelist = growth_plot(replicate_trials=rep_list,
                                    pts_per_hour=pts_per_hour,
                                    plot_start=0, plot_end=49)
    if tracelist:
        fig = go.Figure(data=tracelist, layout=layout)
        plot(fig)
        #pio.write_image(fig,"Figures/si_fig_4_"+strain+".svg",format='svg')
